In [1]:
import pandas as pd
data = pd.read_csv('stock_report.csv', on_bad_lines='skip')

In [2]:
data.tail(5)

,Activity Date,Process Date,Settle Date,Instrument,Description,Trans Code,Quantity,Price,Amount
118,6/10/2024,6/10/2024,6/11/2024,SNOW,Snowflake\nCUSIP: 833445109,Buy,0.000692,$129.88,($0.09)
119,6/10/2024,6/10/2024,6/11/2024,SNOW,Snowflake\nCUSIP: 833445109,Buy,0.153268,$129.88,($19.91)
120,6/10/2024,6/10/2024,6/10/2024,TSLA,Tesla\nCUSIP: 88160R101,REC,0.030300,NaN,NaN
121,6/10/2024,6/10/2024,6/11/2024,NaN,ACH Deposit,ACH,NaN,NaN,$100.00
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
stocks = list(data['Instrument'].value_counts().to_dict().keys())

In [4]:
data = data[data['Instrument'] == 'F']

In [5]:
selected_columns = data[['Instrument', 'Trans Code', 'Quantity', 'Price']]
selected_columns

,Instrument,Trans Code,Quantity,Price
4,F,Buy,3.0,$9.79
5,F,Buy,2.0,$9.77
6,F,Buy,2.0,$9.76
14,F,Buy,2.0,$9.78
19,F,Buy,1.0,$10.88
23,F,Buy,1.0,$10.98
27,F,Buy,2.0,$11.31
28,F,Buy,1.0,$11.23
31,F,Buy,1.0,$11.36
33,F,Buy,2.0,$11.78


In [6]:
# Normalize the prices (remove '$' and convert to float)
selected_columns.loc[:, 'Price'] = selected_columns['Price'].replace('[\$,]', '', regex=True).astype(float)

# Convert 'Sell' transactions to negative quantities
selected_columns.loc[selected_columns['Trans Code'] == 'Sell', 'Quantity'] *= -1

# Format the output
output = selected_columns.apply(lambda row: f"{row['Quantity']} @ {row['Price']}", axis=1)

# Display the result
output

4       3.0 @ 9.79
5       2.0 @ 9.77
6       2.0 @ 9.76
14      2.0 @ 9.78
19     1.0 @ 10.88
23     1.0 @ 10.98
27     2.0 @ 11.31
28     1.0 @ 11.23
31     1.0 @ 11.36
33     2.0 @ 11.78
101    -6.0 @ 12.2
108    6.0 @ 11.87
dtype: object